# Step 1: Set up the LLM app

We use the RAG application created previously and evaluate this in the next few notebooks. We repeat the preprocessing and indexing steps to illustrate the flow from scratch. However, we streamline the earlier flow by using only a single LLM (OpenAI) and vector index (in-memory flat index).

In [1]:
import tqdm
import numpy as np

from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

import sys
sys.path.append("../")
import utils

# Load, preprocess, and chunk the data

We use only 5 samples for speed.

In [2]:
data = utils.load_data(sample_size=5)
documents = utils.preprocess_data(data)
nodes = utils.chunk_documents(documents)
# Load embedding model
embedding_model = utils.get_embedding_model(model_name="BAAI/bge-small-en-v1.5")

Repo card metadata block was not found. Setting CardData to empty.
100%|██████████| 5/5 [00:00<00:00, 3177.02it/s]


Parsing nodes:   0%|          | 0/5 [00:00<?, ?it/s]

Documents before chunking: 5
Documents after chunking: 35


/Users/akashsaravanan/Library/Caches/pypoetry/virtualenvs/genai-bootcamp-4wh1UwyX-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Examine a Document

In [3]:
print(documents[4].text[:500])

 = Plain maskray = 
 
 The plain maskray or brown stingray ( Neotrygon annotata ) is a species of stingray in the family Dasyatidae . It is found in shallow , soft-bottomed habitats off northern Australia . Reaching 24 cm ( 9.4 in ) in width , this species has a diamond-shaped , grayish green pectoral fin disc . Its short , whip-like tail has alternating black and white bands and fin folds above and below . There are short rows of thorns on the back and the base of the tail , but otherwise the s


# Recreate the flat in-memory index

In [4]:
from llama_index.core import VectorStoreIndex
index = VectorStoreIndex(nodes, embed_model=embedding_model, show_progress=True)

Generating embeddings:   0%|          | 0/35 [00:00<?, ?it/s]

# Instantiate the LLM

In [5]:
# OpenAI
from llama_index.llms.openai import OpenAI

# Set up OpenAI client - API key is handled in your .env file
openai_llm = OpenAI(model="gpt-4o", temperature=0.0, max_tokens=2048)

# Setup Prompt Template

In [6]:
from llama_index.core.prompts import PromptTemplate

prompt = """You are a helpful AI assistant that answers questions after carefully reading all the provided context. You always cite sources (document titles) and also quote the relevant snippets. Information may be spread across multiple documents. If the information is not present in any of the contexts, you will say 'I don't know'.
-----
Context: 
{context_str} 
-----
Question: {query_str} 
Answer: `answer`
Source: `source document title`
Relevant Snippet: `snippet`
"""

# Convert prompt into a prompt template that llamaindex can use
prompt = PromptTemplate(template=prompt)

# Test App

In [7]:
user_input = "Where is the plain maskray found?"

In [8]:
openai_query_engine = index.as_query_engine(llm=openai_llm, text_qa_template=prompt, similarity_top_k=3)

In [9]:
print(openai_query_engine.query(user_input))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The plain maskray is found in the continental shelf of northern Australia, from the Wellesley Islands in Queensland to the Bonaparte Archipelago in Western Australia, including the Gulf of Carpentaria and the Timor and Arafura Seas. There are also unsubstantiated reports that its range extends to southern Papua New Guinea.

Source: Plain maskray
Relevant Snippet: "The plain maskray inhabits the continental shelf of northern Australia from the Wellesley Islands in Queensland to the Bonaparte Archipelago in Western Australia, including the Gulf of Carpentaria and the Timor and Arafura Seas. There are unsubstantiated reports that its range extends to southern Papua New Guinea."
